# Imports

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor, Normalize, Resize, CenterCrop
from torchsummary import summary
import pandas as pd
import json

import PIL

from skimage.transform import resize

import gradio as gr

In [2]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

# Preparation of Classes

In [3]:
with open("../../fiftyone/coco-2014/info.json", 'r') as file:
    info = json.load(file)

classes = tuple(info['classes'])

# Model Load

In [4]:
model = torch.load('vgg16_multi-label_coco-2014_model.pth')

In [5]:
model.load_state_dict(torch.load('vgg16_multi-label_coco-2014_model_weights.pth'))

<All keys matched successfully>

In [6]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
model = model.to("cpu")

# Gradio

In [17]:
def predict(img):
    img = Resize(size=256)(img)
    img = CenterCrop(size=224)(img)  # 224 x 224 is required for vgg16
    img_array = ToTensor()(img)
    img_array = Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225])(img_array)
    img_array = img_array.unsqueeze(0)
    prediction = model(img_array)
    predictions_label = prediction > 0.5
    # prediction = nn.Softmax(dim=1)(prediction)
    # prediction = torch.topk(prediction, 10)   <- don't need this because Gradio takes care of this
    predicted_labels = [classes[label] for label in get_index_positions(list(predictions_label[0].cpu().numpy()), 1)]
    # confidences = {classes[i]: float(prediction[0][i].detach()) for i in range(len(classes))}
    confidences = ", ".join(predicted_labels)
    if confidences == "":
        confidences = {classes[i]: float(prediction[0][i].detach()) for i in range(len(classes))}
    else:
        confidences = confidences
    
    return confidences

In [18]:
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(shape=(256, 256), type='pil'),
    # outputs=gr.Label()
    outputs=gr.Label(num_top_classes=3)
)
demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x22d0d8f3b20>, 'http://127.0.0.1:7864/', None)